In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d balraj98/indoor-training-set-its-residestandard -p data
! kaggle datasets download -d balraj98/synthetic-objective-testing-set-sots-reside -p data/SOTS

Saving kaggle.json to kaggle.json
100% 4.73G/4.74G [00:59<00:00, 120MB/s]
100% 4.74G/4.74G [00:59<00:00, 85.9MB/s]
 98% 406M/415M [00:03<00:00, 124MB/s]
100% 415M/415M [00:03<00:00, 114MB/s]


In [ ]:
! unzip /content/data/indoor-training-set-its-residestandard.zip -d /content/data
! unzip /content/data/SOTS/synthetic-objective-testing-set-sots-reside.zip -d /content/data/SOTS


Streaming output truncated to the last 5000 lines.
  inflating: /content/data/trans/68_4.png  
  inflating: /content/data/trans/68_5.png  
  inflating: /content/data/trans/68_6.png  
  inflating: /content/data/trans/68_7.png  
  inflating: /content/data/trans/68_8.png  
  inflating: /content/data/trans/68_9.png  
  inflating: /content/data/trans/690_1.png  
  inflating: /content/data/trans/690_10.png  
  inflating: /content/data/trans/690_2.png  
  inflating: /content/data/trans/690_3.png  
  inflating: /content/data/trans/690_4.png  
  inflating: /content/data/trans/690_5.png  
  inflating: /content/data/trans/690_6.png  
  inflating: /content/data/trans/690_7.png  
  inflating: /content/data/trans/690_8.png  
  inflating: /content/data/trans/690_9.png  
  inflating: /content/data/trans/691_1.png  
  inflating: /content/data/trans/691_10.png  
  inflating: /content/data/trans/691_2.png  
  inflating: /content/data/trans/691_3.png  
  inflating: /content/data/trans/691_4.png  
  inflat

In [ ]:
!pip uninstall -y tensorboard tb-nightly && pip install tb-nightly

Found existing installation: tb-nightly 2.15.0a20230915
Uninstalling tb-nightly-2.15.0a20230915:
  Successfully uninstalled tb-nightly-2.15.0a20230915
  Using cached tb_nightly-2.15.0a20230915-py3-none-any.whl (5.5 MB)


In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Imports

In [ ]:
import torch
import math
import sys
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.backends import cudnn
import torchvision
import torchvision.datasets as datasets
import torchvision.utils as vutils
from torchvision.utils import save_image
from matplotlib import pyplot as plt
import numpy as np
from torch.cuda.amp import GradScaler, autocast
import warnings,os,argparse
import torch.utils.data as data
import torchvision.transforms as tfs
from torchvision.transforms import functional as FF
from matplotlib import pyplot as plt
import functools
from torchvision.utils import make_grid
from torch.utils.tensorboard import SummaryWriter

# Options

In [ ]:
warnings.filterwarnings('ignore')

class options:
  def __init__(self):
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    self.load_model = False
    self.save_model = True
    self.steps = 60001
    self.num_workers = 1
    self.lambda_gen = 2
    self.lambda_identity= 5
    self.lambda_cycle = 10
    self.lambda_pc = 0.1
    self.checkpoint_model_load = "/content/drive/MyDrive/dehaze_v3diglc/trained_models/my_checkpoint_4.pk"
    self.checkpoint_model = "/content/drive/MyDrive/dehaze_v3diglc/trained_models/my_checkpoint_4.pk"
    self.log_dir ="logs"
    self.eval_step =50
    self.lr=0.0001
    self.trainset='its_train'
    self.testsett='its_test'
    self.gps=3
    self.blocks=6
    self.bs=1
    self.crop= True
    self.crop_size=256
    self.ld_input_size = 64


opt = options()


In [ ]:
if not os.path.exists('/content/drive/MyDrive/dehaze_v3diglc/trained_models'):
	os.mkdir('/content/drive/MyDrive/dehaze_v3diglc/trained_models')
if not os.path.exists('/content/drive/MyDrive/dehaze_v3diglc/saved_images'):
	os.mkdir('/content/drive/MyDrive/dehaze_v3diglc/saved_images')
if not os.path.exists('/content/drive/MyDrive/dehaze_v3diglc/test_images'):
	os.mkdir('/content/drive/MyDrive/dehaze_v3diglc/test_images')
if not os.path.exists('/content/drive/MyDrive/dehaze_v3diglc/test_images/best_picks'):
	os.mkdir('/content/drive/MyDrive/dehaze_v3diglc/test_images/best_picks')
if not os.path.exists('/content/drive/MyDrive/dehaze_v3diglc/runs'):
	os.makedirs('/content/drive/MyDrive/dehaze_v3diglc/runs', exist_ok=True)
if not os.path.exists('/content/drive/MyDrive/dehaze_v3diglc/inputs'):
	os.makedirs('/content/drive/MyDrive/dehaze_v3diglc/inputs', exist_ok=True)
if not os.path.exists('/content/drive/MyDrive/dehaze_v3diglc/outputs'):
	os.makedirs('/content/drive/MyDrive/dehaze_v3diglc/outputs', exist_ok=True)

# Metrics

In [ ]:
from math import exp
import math
import numpy as np

import torch.nn.functional as F
from torch.autograd import Variable

from  torchvision.transforms import ToPILImage

def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size // 2) ** 2 / float(2 * sigma ** 2)) for x in range(window_size)])
    return gauss / gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window

def _ssim(img1, img2, window, window_size, channel, size_average=True):
    mu1 = F.conv2d(img1, window, padding=window_size // 2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size // 2, groups=channel)
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2
    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size // 2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size // 2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size // 2, groups=channel) - mu1_mu2
    C1 = 0.01 ** 2
    C2 = 0.03 ** 2
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)


def ssim(img1, img2, window_size=11, size_average=True):
    img1=torch.clamp(img1,min=0,max=1)
    img2=torch.clamp(img2,min=0,max=1)
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)
    return _ssim(img1, img2, window, window_size, channel, size_average)
def psnr(pred, gt):
    pred=pred.clamp(0,1).cpu().numpy()
    gt=gt.clamp(0,1).cpu().numpy()
    imdff = pred - gt
    rmse = math.sqrt(np.mean(imdff ** 2))
    if rmse == 0:
        return 100
    return 20 * math.log10( 1.0 / rmse)

In [ ]:
def save_checkpoint(gen_C, gen_H, disc_C, disc_H, opt_gen, opt_disc, start_step, max_ssim, max_psnr, ssims, psnrs, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "gen_C": gen_C.state_dict(),
        "gen_H": gen_H.state_dict(),
        "disc_C": disc_C.state_dict(),
        "disc_H": disc_H.state_dict(),
        "opt_gen": opt_gen.state_dict(),
        "opt_disc": opt_disc.state_dict(),
        "start_step":start_step,
		    "max_psnr":max_psnr,
        "max_ssim":max_ssim,
        "ssims":ssims,
        "psnrs":psnrs
    }
    torch.save(checkpoint, filename)




# Dehaze Module

In [ ]:
from torch import nn


class DeformConv2d(nn.Module):
    def __init__(self, inc, outc, kernel_size=3, padding=1, stride=1, bias=None, modulation=True):
        """
        Args:
            modulation (bool, optional): If True, Modulated Defomable Convolution (Deformable ConvNets v2).
        """
        super(DeformConv2d, self).__init__()
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride
        self.zero_padding = nn.ZeroPad2d(padding)
        self.conv = nn.Conv2d(inc, outc, kernel_size=kernel_size, stride=kernel_size, bias=bias)

        self.p_conv = nn.Conv2d(inc, 2*kernel_size*kernel_size, kernel_size=3, padding=1, stride=stride)
        nn.init.constant_(self.p_conv.weight, 0)
        self.p_conv.register_backward_hook(self._set_lr)

        self.modulation = modulation
        if modulation:
            self.m_conv = nn.Conv2d(inc, kernel_size*kernel_size, kernel_size=3, padding=1, stride=stride)
            nn.init.constant_(self.m_conv.weight, 0)
            self.m_conv.register_backward_hook(self._set_lr)

    @staticmethod
    def _set_lr(module, grad_input, grad_output):
        grad_input = (grad_input[i] * 0.1 for i in range(len(grad_input)))
        grad_output = (grad_output[i] * 0.1 for i in range(len(grad_output)))

    def forward(self, x):
        offset = self.p_conv(x)
        if self.modulation:
            m = torch.sigmoid(self.m_conv(x))

        dtype = offset.data.type()
        ks = self.kernel_size
        N = offset.size(1) // 2

        if self.padding:
            x = self.zero_padding(x)

        # (b, 2N, h, w)
        p = self._get_p(offset, dtype)

        # (b, h, w, 2N)
        p = p.contiguous().permute(0, 2, 3, 1)
        q_lt = p.detach().floor()
        q_rb = q_lt + 1

        q_lt = torch.cat([torch.clamp(q_lt[..., :N], 0, x.size(2)-1), torch.clamp(q_lt[..., N:], 0, x.size(3)-1)], dim=-1).long()
        q_rb = torch.cat([torch.clamp(q_rb[..., :N], 0, x.size(2)-1), torch.clamp(q_rb[..., N:], 0, x.size(3)-1)], dim=-1).long()
        q_lb = torch.cat([q_lt[..., :N], q_rb[..., N:]], dim=-1)
        q_rt = torch.cat([q_rb[..., :N], q_lt[..., N:]], dim=-1)

        # clip p
        p = torch.cat([torch.clamp(p[..., :N], 0, x.size(2)-1), torch.clamp(p[..., N:], 0, x.size(3)-1)], dim=-1)

        # bilinear kernel (b, h, w, N)
        g_lt = (1 + (q_lt[..., :N].type_as(p) - p[..., :N])) * (1 + (q_lt[..., N:].type_as(p) - p[..., N:]))
        g_rb = (1 - (q_rb[..., :N].type_as(p) - p[..., :N])) * (1 - (q_rb[..., N:].type_as(p) - p[..., N:]))
        g_lb = (1 + (q_lb[..., :N].type_as(p) - p[..., :N])) * (1 - (q_lb[..., N:].type_as(p) - p[..., N:]))
        g_rt = (1 - (q_rt[..., :N].type_as(p) - p[..., :N])) * (1 + (q_rt[..., N:].type_as(p) - p[..., N:]))

        # (b, c, h, w, N)
        x_q_lt = self._get_x_q(x, q_lt, N)
        x_q_rb = self._get_x_q(x, q_rb, N)
        x_q_lb = self._get_x_q(x, q_lb, N)
        x_q_rt = self._get_x_q(x, q_rt, N)

        # (b, c, h, w, N)
        x_offset = g_lt.unsqueeze(dim=1) * x_q_lt + \
                   g_rb.unsqueeze(dim=1) * x_q_rb + \
                   g_lb.unsqueeze(dim=1) * x_q_lb + \
                   g_rt.unsqueeze(dim=1) * x_q_rt

        # modulation
        if self.modulation:
            m = m.contiguous().permute(0, 2, 3, 1)
            m = m.unsqueeze(dim=1)
            m = torch.cat([m for _ in range(x_offset.size(1))], dim=1)
            x_offset *= m

        x_offset = self._reshape_x_offset(x_offset, ks)
        out = self.conv(x_offset)

        return out

    def _get_p_n(self, N, dtype):
        p_n_x, p_n_y = torch.meshgrid(
            torch.arange(-(self.kernel_size-1)//2, (self.kernel_size-1)//2+1),
            torch.arange(-(self.kernel_size-1)//2, (self.kernel_size-1)//2+1))
        # (2N, 1)
        p_n = torch.cat([torch.flatten(p_n_x), torch.flatten(p_n_y)], 0)
        p_n = p_n.view(1, 2*N, 1, 1).type(dtype)

        return p_n

    def _get_p_0(self, h, w, N, dtype):
        p_0_x, p_0_y = torch.meshgrid(
            torch.arange(1, h*self.stride+1, self.stride),
            torch.arange(1, w*self.stride+1, self.stride))
        p_0_x = torch.flatten(p_0_x).view(1, 1, h, w).repeat(1, N, 1, 1)
        p_0_y = torch.flatten(p_0_y).view(1, 1, h, w).repeat(1, N, 1, 1)
        p_0 = torch.cat([p_0_x, p_0_y], 1).type(dtype)

        return p_0

    def _get_p(self, offset, dtype):
        N, h, w = offset.size(1)//2, offset.size(2), offset.size(3)

        # (1, 2N, 1, 1)
        p_n = self._get_p_n(N, dtype)
        # (1, 2N, h, w)
        p_0 = self._get_p_0(h, w, N, dtype)
        p = p_0 + p_n + offset
        return p

    def _get_x_q(self, x, q, N):
        b, h, w, _ = q.size()
        padded_w = x.size(3)
        c = x.size(1)
        # (b, c, h*w)
        x = x.contiguous().view(b, c, -1)

        # (b, h, w, N)
        index = q[..., :N]*padded_w + q[..., N:]  # offset_x*w + offset_y
        # (b, c, h*w*N)
        index = index.contiguous().unsqueeze(dim=1).expand(-1, c, -1, -1, -1).contiguous().view(b, c, -1)

        x_offset = x.gather(dim=-1, index=index).contiguous().view(b, c, h, w, N)

        return x_offset

    @staticmethod
    def _reshape_x_offset(x_offset, ks):
        b, c, h, w, N = x_offset.size()
        x_offset = torch.cat([x_offset[..., s:s+ks].contiguous().view(b, c, h, w*ks) for s in range(0, N, ks)], dim=-1)
        x_offset = x_offset.contiguous().view(b, c, h*ks, w*ks)

        return x_offset

In [ ]:
import torch.nn as nn

def default_conv(in_channels, out_channels, kernel_size, bias=True):
    return nn.Conv2d(in_channels, out_channels, kernel_size, padding=(kernel_size//2), bias=bias)

class PALayer(nn.Module):
    def __init__(self, channel):
        super(PALayer, self).__init__()
        self.pa = nn.Sequential(
                nn.Conv2d(channel, channel // 4, kernel_size=3, stride=1, dilation=2, padding=2, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 4, channel // 8, kernel_size=3, stride=1, dilation=1, padding=1, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 8, 1, kernel_size=1, stride=1, dilation=1, padding=0, bias=True),
                nn.Sigmoid()
        )
    def forward(self, x):
        y = self.pa(x)
        return x * y

class CALayer(nn.Module):
    def __init__(self, channel):
        super(CALayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.ca = nn.Sequential(
                nn.Conv2d(channel, channel // 8, 1, padding=0, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 8, channel, 1, padding=0, bias=True),
                nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.ca(y)
        return x * y

class DCNBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(DCNBlock, self).__init__()
        self.dcn = DeformConv2d(in_channel, out_channel, kernel_size=3, padding=1, stride=1).cuda()
    def forward(self, x):
        return self.dcn(x)

class Block(nn.Module):
    def __init__(self, conv, dim, kernel_size,):
        super(Block, self).__init__()
        self.conv1=conv(dim, dim, kernel_size, bias=True)
        self.act1=nn.LeakyReLU(0.2,inplace=True)
        self.conv2=conv(dim,dim,kernel_size,bias=True)
        self.calayer=CALayer(dim)
        self.palayer=PALayer(dim)
    def forward(self, x):
        res=self.act1(self.conv1(x))
        res=res+x
        res=self.conv2(res)
        res=self.calayer(res)
        res=self.palayer(res)
        res += x
        return res

class DehazeModule(nn.Module):
    def __init__(self,blocks = opt.blocks,conv=default_conv,io_channel = 256):
        super(DehazeModule, self).__init__()
        self.dim= io_channel
        kernel_size = 3
        modules = [ Block(conv, self.dim, kernel_size)  for _ in range(blocks)]
        self.gp = nn.Sequential(*modules)
        self.dcn_block = DCNBlock(self.dim, self.dim)


    def forward(self, x):

        res = self.gp(x)
        res_dcn1 = self.dcn_block(res)
        res_dcn2 = self.dcn_block(res_dcn1)

        return res_dcn2



# Generator

In [ ]:

class Mix(nn.Module):
    def __init__(self, m=-0.80):
        super(Mix, self).__init__()
        w = torch.nn.Parameter(torch.FloatTensor([m]), requires_grad=True)
        w = torch.nn.Parameter(w, requires_grad=True)
        self.w = w
        self.mix_block = nn.Sigmoid()

    def forward(self, fea1, fea2):
        mix_factor = self.mix_block(self.w)
        out = fea1 * mix_factor.expand_as(fea1) + fea2 * (1 - mix_factor.expand_as(fea2))
        return out

class ConvBlock(nn.Module):

    def __init__(self, in_channels, out_channels, down=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs)
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)


class Generator(nn.Module):
    def __init__(self,img_channels=3,out=3):
        super(Generator,self).__init__()
        self.down1 = nn.Sequential(nn.Conv2d(img_channels, 64, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),nn.InstanceNorm2d(64),nn.ReLU(inplace=True))
        self.down2 = ConvBlock(64, 128, kernel_size=3, stride=2, padding=1)
        self.down3 = ConvBlock(128, 256, kernel_size=3, stride=2, padding=1)


        self.dehaze_block = DehazeModule()


        self.up1 = ConvBlock(256, 128, down=False, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up2 = ConvBlock(128, 64, down=False, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up3 = nn.Sequential(nn.Conv2d(64, img_channels, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),nn.Tanh())

        self.mix1 = Mix(m=-1)
        self.mix2 = Mix(m=-0.6)

    def forward(self, input):
        x_down1 = self.down1(input)
        x_down2 = self.down2(x_down1)
        x_down3 = self.down3(x_down2)

        x_dehaze = self.dehaze_block(x_down3)

        x_out_mix = self.mix1(x_down3, x_dehaze)
        x_up1 = self.up1(x_out_mix)
        x_up1_mix = self.mix2(x_down2, x_up1)
        x_up2 = self.up2(x_up1_mix)
        out = self.up3(x_up2)

        return out



# Discriminator


In [ ]:
import random
import torch

def gen_hole_area(size, mask_size):
    """
    * inputs:
        - size (sequence, required)
                A sequence of length 2 (W, H) is assumed.
                (W, H) is the size of hole area.
        - mask_size (sequence, required)
                A sequence of length 2 (W, H) is assumed.
                (W, H) is the size of input mask.
    * returns:
            A sequence used for the input argument 'hole_area' for function 'gen_input_mask'.
    """
    mask_w, mask_h = mask_size
    harea_w, harea_h = size
    offset_x = random.randint(0, mask_w - harea_w)
    offset_y = random.randint(0, mask_h - harea_h)
    return ((offset_x, offset_y), (harea_w, harea_h))


def crop(x, area):
    """
    * inputs:
        - x (torch.Tensor, required)
                A torch tensor of shape (N, C, H, W) is assumed.
        - area (sequence, required)
                A sequence of length 2 ((X, Y), (W, H)) is assumed.
                sequence[0] (X, Y) is the left corner of an area to be cropped.
                sequence[1] (W, H) is its width and height.
    * returns:
            A torch tensor of shape (N, C, H, W) cropped in the specified area.
    """
    xmin, ymin = area[0]
    w, h = area[1]
    return x[:, :, ymin: ymin + h, xmin: xmin + w]

In [ ]:
class DiscModule(nn.Module):
    def __init__(self,in_channels=3,out_channels=1):
        super(DiscModule, self).__init__()

        norm_layer = functools.partial(nn.InstanceNorm2d, affine=False, track_running_stats=False)

        model =    [
                        nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1),
                        nn.LeakyReLU(0.2,True),

                        nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=True),
                        norm_layer(128),
                        nn.LeakyReLU(0.2, True),

                        nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=True),
                        norm_layer(256),
                        nn.LeakyReLU(0.2, True),

                        nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1, bias=True),
                        norm_layer(512),
                        nn.LeakyReLU(0.2, True),

                        nn.Conv2d(512, out_channels, kernel_size=4, stride=1, padding=1)
                    ]
        self.model = nn.Sequential(*model)

    def forward(self, input):

        return torch.flatten(self.model(input))


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model_ld = DiscModule()
        self.model_gd = DiscModule()

        self.act1 = nn.Sigmoid()

    def forward(self, x):
        x_c1 = crop(x,gen_hole_area((opt.ld_input_size, opt.ld_input_size),(x.shape[3], x.shape[2])))
        x_c2 = crop(x,gen_hole_area((opt.ld_input_size, opt.ld_input_size),(x.shape[3], x.shape[2])))
        x_c3 = crop(x,gen_hole_area((opt.ld_input_size, opt.ld_input_size),(x.shape[3], x.shape[2])))

        x_gd = self.model_ld(x)
        x_ld1 = self.model_gd(x_c1)
        x_ld2 = self.model_gd(x_c2)
        x_ld3 = self.model_gd(x_c3)

        out = self.act1((torch.cat((x_gd, x_ld1, x_ld2, x_ld3),-1)))
        return out

# Dataset

In [ ]:
sys.path.append('.')
sys.path.append('..')
import numpy as np
import torch
import random
from PIL import Image

BS=opt.bs
print(BS)
if opt.crop:
    crop_size=opt.crop_size


class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
        return tensor

def tensorShow(tensor,titles=None):

        fig=plt.figure()
        img = make_grid(tensor)
        npimg = img.numpy()
        ax = fig.add_subplot(211)
        ax.imshow(np.transpose(npimg, (1, 2, 0)))
        ax.set_title(titles)
        plt.show()




class RESIDE_Dataset(data.Dataset):
    def __init__(self,path,train,size=crop_size,format='.png'):

        super(RESIDE_Dataset,self).__init__()
        self.size=size
        print('crop size',size)
        self.train=train
        self.format=format
        self.haze_imgs_dir=os.listdir(os.path.join(path,'hazy'))
        self.haze_imgs=[os.path.join(path,'hazy',img) for img in self.haze_imgs_dir]
        self.clear_dir=os.path.join(path,'clear')
        self.transforms = self.get_transforms()
        self.transformstest = self.get_transforms_test()

    def __getitem__(self, index):

        haze=Image.open(self.haze_imgs[index])
        img=self.haze_imgs[index]
        id=img.split('/')[-1].split('_')[0]
        clear_name=id+self.format
        clear=Image.open(os.path.join(self.clear_dir,clear_name))
        if self.train:
          hazeimage = self.transforms(haze.convert("RGB"))
          clearimage = self.transforms(clear.convert("RGB"))
        else :
          w, h = haze.size
          clear = clear.resize((w-(w%4),h-(h%4)))
          haze = haze.resize((w-(w%4),h-(h%4)))
          hazeimage = self.transformstest(haze.convert("RGB"))
          clearimage = self.transformstest(clear.convert("RGB"))


        return hazeimage,clearimage

    def get_transforms(self, resize_to=350, interpolation=Image.BICUBIC):
        all_transforms=[]
        crop_s=opt.crop_size
        all_transforms.append(tfs.Resize(size=(resize_to,resize_to), interpolation=interpolation))
        all_transforms.append(tfs.RandomCrop(crop_s))
        all_transforms.append(tfs.ToTensor())
        all_transforms.append(tfs.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) ))
        return tfs.Compose(all_transforms)

    def get_transforms_test(self):
        all_transforms=[]
        all_transforms.append(tfs.ToTensor())
        all_transforms.append(tfs.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)))
        return tfs.Compose(all_transforms)

    def __len__(self):
        return len(self.haze_imgs)

pwd=os.getcwd()
print(pwd)
path='/content/data'  #path to your 'data' folder

ITS_train_loader=DataLoader(dataset=RESIDE_Dataset('/content/data',train=True,size=opt.crop_size),batch_size=opt.bs,shuffle=True,num_workers= 4,pin_memory=True)
ITS_test_loader=DataLoader(dataset=RESIDE_Dataset('/content/data/SOTS/indoor',train=False,size='whole img'),batch_size=1,shuffle=False,pin_memory=True)

class Eval_Dataset(data.Dataset):
    def __init__(self,path,train,size=crop_size,format='.png'):

        super(Eval_Dataset,self).__init__()
        self.size=size
        print('crop size',size)
        self.train=train
        self.format=format
        self.haze_imgs_dir=os.listdir(path)
        self.haze_imgs=[os.path.join(path,img) for img in self.haze_imgs_dir]
        self.transformstest = self.get_transforms_test()

    def __getitem__(self, index):

        haze = Image.open(self.haze_imgs[index])
        w, h = haze.size
        haze = haze.resize((w-(w%4),h-(h%4)))
        hazeimage = self.transformstest(haze.convert("RGB"))
        return hazeimage

    def get_transforms_test(self, interpolation=Image.BICUBIC):
        all_transforms=[]
        all_transforms.append(tfs.ToTensor())
        all_transforms.append(tfs.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) ))
        return tfs.Compose(all_transforms)

    def __len__(self):
        return len(self.haze_imgs)

pwd=os.getcwd()
print(pwd)




1
/content
crop size 256
crop size whole img
/content


In [ ]:
! ls

data  drive  kaggle.json  sample_data


In [ ]:
from torchvision import models
from torch.autograd import Variable
from PIL import Image
from collections import namedtuple

class Vgg16(torch.nn.Module):
    def __init__(self):
        super(Vgg16, self).__init__()
        features = list(models.vgg16(pretrained=True).features)[:]
        self.features = nn.ModuleList(features).eval()

    def forward(self, x):
        results = []
        for ii, model in enumerate(self.features):
            x = model(x)
            # print(x.shape)
            if ii in {16,30}:
                results.append(x)

        vgg_outputs = namedtuple("VggOutputs", ['mp_2', 'mp_5'])
        return vgg_outputs(*results)

# Train

In [ ]:
def lr_schedule_cosdecay(t,T = opt.steps,init_lr=opt.lr):
	lr=0.5*(1+math.cos(t*math.pi/T))*init_lr
	return lr

vgg = Vgg16().eval()
vgg.to(opt.device)

def train_fn(disc_C, disc_H, gen_H, gen_C, train_loader, test_loader, opt_disc, opt_gen, l1, mse, just_eval=False):


    start_step = 0
    max_ssim = 0
    max_psnr = 0
    ssims = []
    psnrs = []
    g_scaler = GradScaler()
    d_scaler = GradScaler()
    writer = SummaryWriter(f'/content/drive/MyDrive/dehaze_v3diglc/runs')
    if opt.load_model:
        print("=> Loading checkpoint")
        checkpoint = torch.load(opt.checkpoint_model_load, map_location=opt.device)
        gen_C.load_state_dict(checkpoint["gen_C"])
        gen_H.load_state_dict(checkpoint["gen_H"])
        disc_C.load_state_dict(checkpoint["disc_C"])
        disc_H.load_state_dict(checkpoint["disc_H"])
        opt_gen.load_state_dict(checkpoint["opt_gen"])
        opt_disc.load_state_dict(checkpoint["opt_disc"])
        start_step = checkpoint['start_step']
        max_ssim = checkpoint['max_ssim']
        max_psnr = checkpoint['max_psnr']
        psnrs = checkpoint['psnrs']
        ssims = checkpoint['ssims']


    if(just_eval):
      print(f'just eval ---')
      eval_loader=DataLoader(dataset=Eval_Dataset('/content/drive/MyDrive/dehaze_v3diglc/inputs',train=False,size='whole img'),batch_size=1,shuffle=False,pin_memory=True)
      Eval(gen_C,eval_loader)
      return

    if(opt.load_model):

        print(f'starting training at {start_step} ---')

    else :
        print('train from scratch ---')

    # C_reals = 0
    # C_fakes = 0
    loop = tqdm(range(start_step+1,opt.steps))

    train_iterator = iter(train_loader)

    for idx in loop:


        lr=lr_schedule_cosdecay(idx)
        for param_group in opt_gen.param_groups:
            param_group["lr"] = lr


        haze, clean = next(train_iterator)
        haze = haze.to(opt.device)
        clean = clean.to(opt.device)
        # with autocast():
        fake_clean = gen_C(haze)
        D_C_real = disc_C(clean)
        D_C_fake = disc_C(fake_clean.detach())
        # C_reals += D_C_real.mean().item()
        # C_fakes += D_C_fake.mean().item()
        D_C_real_loss = mse(D_C_real, torch.ones_like(D_C_real))
        D_C_fake_loss = mse(D_C_fake, torch.zeros_like(D_C_fake))
        D_C_loss = D_C_real_loss + D_C_fake_loss

        fake_haze = gen_H(clean)
        D_H_real = disc_H(haze)
        D_H_fake = disc_H(fake_haze.detach())
        D_H_real_loss = mse(D_H_real, torch.ones_like(D_H_real))
        D_H_fake_loss = mse(D_H_fake, torch.zeros_like(D_H_fake))
        D_H_loss = D_H_real_loss + D_H_fake_loss

        D_loss = (D_H_loss + D_C_loss)/2

        # opt_disc.zero_grad()
        # d_scaler.scale(D_loss).backward()
        # d_scaler.step(opt_disc)
        # d_scaler.update()
        opt_disc.zero_grad()
        D_loss.backward()
        opt_disc.step()

        # with autocast():
        # adversarial loss for both generators
        D_C_fake = disc_C(fake_clean)
        D_H_fake = disc_H(fake_haze)
        loss_G_C = mse(D_C_fake, torch.ones_like(D_C_fake))
        loss_G_H = mse(D_H_fake, torch.ones_like(D_H_fake))

        # cycle loss
        cycle_haze = gen_H(fake_clean)
        cycle_clean = gen_C(fake_haze)
        cycle_haze_loss = l1(haze, cycle_haze)
        cycle_clean_loss = l1(clean, cycle_clean)

        # identity loss (remove these for efficiency if you set lambda_identity=0)
        identity_haze = gen_H(haze)
        identity_clean = gen_C(clean)
        identity_haze_loss = l1(haze, identity_haze)
        identity_clean_loss = l1(clean, identity_clean)

        # perpectual cyclic consistency loss
        scaler = tfs.Resize((224, 224))
        for param in vgg.parameters():
          param.requires_grad = False

        fhaze_sc = vgg(scaler(haze))
        fcycle_haze_sc = vgg(scaler(cycle_haze))
        fclean_sc = vgg(scaler(clean))
        fcycle_clean_sc = vgg(scaler(cycle_clean))

        pcm1 = mse(fhaze_sc.mp_2,fcycle_haze_sc.mp_2)
        pcm2 = mse(fhaze_sc.mp_5,fcycle_haze_sc.mp_5)
        pcm3 = mse(fclean_sc.mp_2,fcycle_clean_sc.mp_2)
        pcm4 = mse(fclean_sc.mp_5,fcycle_clean_sc.mp_5)

        perceptual_loss = (pcm1 + pcm2 + pcm3 + pcm4)

        # print(perceptual_loss)
        # add all togethor
        G_loss = (
            loss_G_C * opt.lambda_gen
            + loss_G_H * opt.lambda_gen
            + cycle_haze_loss * opt.lambda_cycle
            + cycle_clean_loss * opt.lambda_cycle
            + identity_clean_loss * opt.lambda_identity
            + identity_haze_loss * opt.lambda_identity
            + perceptual_loss * opt.lambda_pc
        )

        opt_gen.zero_grad()
        G_loss.backward()
        opt_gen.step()
        writer.add_scalar('data/loss_G',G_loss,idx)
        # writer.add_scalar('data/loss_Gen_C',loss_G_C,idx)
        # writer.add_scalar('data/loss_cycle_clean',cycle_clean_loss,idx)
        # writer.add_scalar('data/loss_clean_identity',identity_clean_loss,idx)
        writer.add_scalar('data/loss_Disc',D_loss,idx)
        writer.add_scalar('data/loss_P_C',perceptual_loss,idx)


        if idx % opt.eval_step == 0:
            # save_image(fake_clean*0.5+0.5, f"saved_images/clean_{idx}.png")
            # save_image(fake_haze*0.5+0.5, f"saved_images/haze_{idx}.png")
            unorm = UnNormalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))
            # img_grid = vutils.make_grid([torch.squeeze(haze.cpu()[0]), torch.squeeze(
            #     fake_clean.cpu()[0]), torch.squeeze(clean.cpu()[0]), torch.squeeze(fake_haze.cpu()[0])])
            img_grid = vutils.make_grid([torch.squeeze(unorm(haze.detach()).cpu()[0]), torch.squeeze(
                unorm(fake_clean.detach()).cpu()[0]), torch.squeeze(unorm(clean.detach()).cpu()[0]), torch.squeeze(unorm(fake_haze.detach()).cpu()[0])])
            save_image(img_grid, f"/content/drive/MyDrive/dehaze_v3diglc/saved_images/img_{idx}.png")
            # save_image(img_grid, f"saved_images/img_{epoch}_{idx}.png")
            # save_image(fake_clean, f"saved_images/clean_{epoch}_{idx}.png")
            # save_image(fake_z, f"saved_images/haze_{epoch}_{idx}.png")
            # tensorShow(clean.detach().cpu())
            # tensorShow(unorm(haze).detach().cpu())

        if idx % (10*opt.eval_step) == 0 and idx != 0 :
            with torch.no_grad():
                ssim_eval, psnr_eval = test(gen_C, test_loader, max_psnr, max_ssim, idx)
            print(f'\n iter :{idx} |ssim:{ssim_eval:.4f}| psnr:{psnr_eval:.4f}')

            writer.add_scalar('data/ssim',ssim_eval,idx)
            writer.add_scalar('data/psnr',psnr_eval,idx)
            writer.add_scalars('group',{
                'ssim':ssim_eval,
                'psnr':psnr_eval,
                'loss':G_loss
            },idx)



            ssims.append(ssim_eval)
            psnrs.append(psnr_eval)
            # if ssim_eval > max_ssim or psnr_eval > max_psnr :
            if True :
                print("sim eval")
                max_ssim=max(max_ssim,ssim_eval)
                max_psnr=max(max_psnr,psnr_eval)
                save_checkpoint(gen_C, gen_H, disc_C, disc_H, opt_gen, opt_disc, idx,
                        max_ssim, max_psnr, ssims, psnrs, filename=opt.checkpoint_model)

                print(f'\n model saved at :{idx} | max_psnr:{max_psnr:.4f}|max_ssim:{max_ssim:.4f}')

        loop.set_postfix(iter = idx,G_loss = G_loss.item(),D_loss = D_loss.item(),cycle_H_loss = cycle_haze_loss.item(),cycle_C_loss = cycle_clean_loss.item(),G_C_loss = loss_G_C.item(),G_H_loss = loss_G_H.item(),I_H_loss = identity_haze_loss.item(),I_loss = identity_clean_loss.item(),PC_loss = perceptual_loss.item() , max_ssim = max_ssim, max_psnr = max_psnr)

    if opt.save_model:
      save_checkpoint(gen_C, gen_H, disc_C, disc_H, opt_gen, opt_disc, idx, max_ssim, max_psnr, ssims, psnrs, filename=opt.checkpoint_model)



def test(gen_C, test_loader, max_psnr, max_ssim, idx):
    gen_C.eval()
    torch.cuda.empty_cache()
    ssims = []
    psnrs = []
    s = True
    val_loop = tqdm(test_loader, leave=True)
    for i, (inputs, targets) in enumerate(val_loop):
        inputs = inputs.to(opt.device)
        targets = targets.to(opt.device)
        pred = gen_C(inputs)
        unorm = UnNormalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5))
        ts = vutils.make_grid([torch.squeeze(unorm(inputs.detach()).cpu()[0]), torch.squeeze(unorm(targets.detach()).cpu()[0]), torch.squeeze(unorm(pred.detach()).cpu()[0])])

        # ts = vutils.make_grid([torch.squeeze(inputs.cpu()[0]), torch.squeeze(targets.cpu()[0]), torch.squeeze(pred.cpu()[0])])
        vutils.save_image(ts, f'/content/drive/MyDrive/dehaze_v3diglc/test_images/{i}_testimg.png')
        ssim1 = ssim(unorm(pred.detach()), unorm(targets.detach())).item()
        psnr1 = psnr(unorm(pred.detach()), unorm(targets.detach()))
        ssims.append(ssim1)
        psnrs.append(psnr1)
        if (psnr1 > max_psnr and ssim1 > max_ssim) and s:
            vutils.save_image(ts, f'/content/drive/MyDrive/dehaze_v3diglc/test_images/best_picks/{idx}_ps_{psnr1:.4}_{ssim1:.4}.png')
            s = False
        val_loop.set_postfix(ssim = ssim1, psnr = psnr1)
    return np.mean(ssims) ,np.mean(psnrs)



def Eval(gen_C, eval_loader):
    gen_C.eval()
    torch.cuda.empty_cache()
    s = True
    val_loop = tqdm(eval_loader, leave=True)
    for i, (inputs) in enumerate(val_loop):
        inputs = inputs.to(opt.device)
        pred = gen_C(inputs)
        unorm = UnNormalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5))
        vutils.save_image(torch.squeeze(unorm(pred.detach()).cpu()[0]), f'/content/drive/MyDrive/dehaze_v3diglc/outputs/{i}_testimg.png')



In [ ]:

def main():

    disc_C = Discriminator().to(opt.device)
    disc_H = Discriminator().to(opt.device)
    gen_H = Generator(img_channels=3).to(opt.device)
    gen_C = Generator(img_channels=3).to(opt.device)
    if opt.device=='cuda':
        disc_C=torch.nn.DataParallel(disc_C)
        disc_H=torch.nn.DataParallel(disc_H)
        gen_H=torch.nn.DataParallel(gen_H)
        gen_C=torch.nn.DataParallel(gen_C)
        cudnn.benchmark=True

    opt_disc = optim.Adam(
        list(disc_C.parameters()) + list(disc_H.parameters()),
        lr=opt.lr,
        betas=(0.5, 0.999),
    )

    opt_gen = optim.Adam(
        list(gen_H.parameters()) + list(gen_C.parameters()),
        lr=opt.lr,
        betas=(0.5, 0.999),
    )

    L1 = nn.L1Loss()
    mse = nn.MSELoss()
    train_fn(disc_C, disc_H, gen_H, gen_C, ITS_train_loader, ITS_test_loader,
                 opt_disc, opt_gen, L1, mse, just_eval=False)

# TensorBoard

In [ ]:
%tensorboard --logdir '/content/drive/MyDrive/dehaze_v3diglc/runs' --load_fast true


Reusing TensorBoard on port 6006 (pid 6112), started 0:07:46 ago. (Use '!kill 6112' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
main()

train from scratch ---


100%|██████████| 500/500 [05:25<00:00,  1.53it/s, psnr=22.6, ssim=0.894]



 iter :500 |ssim:0.9195| psnr:29.7480
sim eval
=> Saving checkpoint


  1%|          | 500/60000 [12:56<1639:33:24, 99.20s/it, D_loss=0.188, G_C_loss=0.45, G_H_loss=0.654, G_loss=12.6, I_H_loss=0.474, I_loss=0.157, PC_loss=6.32, cycle_C_loss=0.193, cycle_H_loss=0.472, iter=500, max_psnr=29.7, max_ssim=0.919]


 model saved at :500 | max_psnr:29.7480|max_ssim:0.9195


  2%|▏         | 913/60000 [18:56<14:06:32,  1.16it/s, D_loss=0.471, G_C_loss=0.24, G_H_loss=0.506, G_loss=5.8, I_H_loss=0.0848, I_loss=0.16, PC_loss=5.74, cycle_C_loss=0.159, cycle_H_loss=0.0915, iter=913, max_psnr=29.7, max_ssim=0.919]